In [1]:
# [1] COLLECT ALL GROUND TRUTH DATA
import collections
from plot_overlap_ratios import plot_ratio_success
from algorithm_output_parser import process_algorithm_output
from algorithm_output_parser import toboggan_output_parser
from algorithm_output_parser import catfish_output_parser

from load_all_data import get_all_data


show_flag = False


# [1] get all data
all_path_info, all_toboggan_data = get_all_data( [0,1,2] )
froots = ['human', 'mouse', 'zebra']

Loading human
Loading mouse
Loading zebra
Done with get_all_data



In [11]:
gt_pathset_dict = []
all_catfish_paths = []
all_toboggan_paths = []
all_stats = [collections.defaultdict(list), collections.defaultdict(list), collections.defaultdict(list)]
indices_identical = [collections.defaultdict(list), collections.defaultdict(list), collections.defaultdict(list)]
indices_different = [collections.defaultdict(list), collections.defaultdict(list), collections.defaultdict(list)]
indices_timeout = [collections.defaultdict(list), collections.defaultdict(list), collections.defaultdict(list)]

for which_dataset in [0,1,2]:
    #     # FILTER ALGORITHM OUTPUTS FOR RELEVANT DATA
    froot = froots[which_dataset]
    gt_pathset_dict = all_path_info['groundtruth'][froot]
    all_catfish_paths = all_path_info['catfish'][froot]
    all_toboggan_paths = all_path_info['toboggan'][froot]
    
    non_opt_groundtruth_list = []
    for key, path in gt_pathset_dict.items():
        kpath = all_toboggan_paths[key]
        if len(kpath) != len(path):
            non_opt_groundtruth_list.append(key)
    for badkey in non_opt_groundtruth_list:
        gt_pathset_dict.pop(badkey, None)
            
    
    j = which_dataset
    all_stats[j][0], indices_identical[j][0], indices_different[j][0], indices_timeout[j][0] = process_algorithm_output(gt_pathset_dict, all_catfish_paths)
    all_stats[j][1], indices_identical[j][1], indices_different[j][1], indices_timeout[j][1] = process_algorithm_output(gt_pathset_dict, all_toboggan_paths)
    

Num timeouts = 0
Num timeouts = 0
Num timeouts = 0
Num timeouts = 0
Num timeouts = 0
Num timeouts = 0


In [31]:
alg_names = ['catfish', 'toboggan']
total_num = 0
identical_num = [0, 0]
for j in [0,1,2]:
    froot = froots[j]
    print("For {},".format(froot))
    total_num += len(all_stats[j][0])
    for which_alg in [0,1]:
        alg_name = alg_names[which_alg]
        print("{} percent identical is: {}".format( alg_name, len(indices_identical[j][which_alg])/len(all_stats[j][which_alg])))

        identical_num[which_alg] += len(indices_identical[j][which_alg])
        
for which_alg in [0,1]:
    alg_name = alg_names[which_alg]
    print("TOTALS: {} percent identical is: {}".format( alg_name, identical_num[which_alg]/total_num))


For human,
catfish percent identical is: 0.9690024012224405
toboggan percent identical is: 0.931298463407458
For mouse,
catfish percent identical is: 0.9690272542303694
toboggan percent identical is: 0.9017920735453886
For zebra,
catfish percent identical is: 0.980496784259145
toboggan percent identical is: 0.8973140426296118
TOTALS: catfish percent identical is: 0.97255981960712
TOTALS: toboggan percent identical is: 0.9111843724200748


In [28]:
import matplotlib.pyplot as plt
import numpy
figurename = 'size-vs-ratio-identical-'

def plot_ratios(figurename, froot, all_stats, indices_identical, indices_timeout, show_flag=True):
    plt.close()
    plt.clf()
    plt.figure(figsize=(4,3))

    alg_names = ['catfish', 'toboggan']
    colors = ['red', 'blue']

    def get_width(num_datapoints):
        return 0.18*numpy.log10(num_datapoints)

    fig, ax1 = plt.subplots()
    
    fig_alg = {}
    max_label = 0

    for which_alg in range(len(alg_names)):
        print(" {} with # data points {} ".format(alg_names[which_alg], len(all_stats[which_alg])) )
        # all_stats.append( [len(gt_pathset), len(catfish_pathset), this_jaccard_value, this_intersect, gap] )
        stats = all_stats[which_alg]
        inds_identical = indices_identical[which_alg]
        inds_timeouts = indices_timeout[which_alg]

        data1 = numpy.array(stats[:,0])  # size of ground truth pathset
        data2 = numpy.array(stats[inds_identical, 0])  # size of ground truth pathset when algorithm identical

        data1_counter = collections.Counter([ x[0] for x in data1 ])
        data2_counter = collections.Counter([ x[0] for x in data2 ])

        data1_counter = collections.Counter([ x[0] for x in data1 ])
        data2_counter = collections.Counter([ x[0] for x in data2 ])

        data = []
        soln_sizes = []
        num_datapoints = []
        widths = []
        for soln_size, x in data1_counter.items():
            y = data2_counter[soln_size]
            if x != 0:
                data.append(y/x)
                num_datapoints.append(int(x))
                soln_sizes.append(int(soln_size))
                widths.append(get_width(x))
                if y != 0:
                    max_label = max( max_label, int(soln_size) )

        ax1.scatter(soln_sizes, data, color=colors[which_alg], label=alg_names[which_alg], alpha=0.5)
    
    axis_font_size = 15
    axis_font = {'fontname':'DejaVu Sans', 'size':'{}'.format(axis_font_size)}
    title_font = {'fontname':'DejaVu Sans', 'size':'16'}
    xtick_font = 12 #{'fontname':'DejaVu Sans', 'size':'11'}
    ytick_font = 13 #{'fontname':'DejaVu Sans', 'size':'12'}
    #         plt.ylabel("Frequency", **axis_font)
    #         plt.gca().set_yticklabels([10**x for x in range(0, 6)], **ytick_font)
    #     plt.title("dataset: {}".format(froot), **title_font)
    #     plt.xlabel("time", **axis_font)
    #     plt.gca().set_xticklabels([10**x for x in range(-4, 5)], **xtick_font)

    plt.title("Pathset overlap on {}".format(froot), **title_font)
    plt.xlabel("Ground truth decomposition size", **axis_font)
    xlims = plt.xlim()
    # Determine xlims
    xlimmin = xlims[0]
    xlimmax = 12
    ax1.set_xlim([xlimmin, xlimmax])
    
    ax1.set_ylim([0,1])
    ax1.set_ylabel("Proportion of instances: output = GT", color='purple', **axis_font)
    ax1.tick_params('y', colors='purple', labelsize=ytick_font)

    # get all ground truth sizes
    soln_sizes = [ (int(soln_size), int(count)) for soln_size, count in \
                  collections.Counter([ x[0] for x in \
                                       numpy.array(all_stats[0][:,0]) ]).items() if count != 0 ]         
    soln_sizes, num_datapoints = zip(*soln_sizes)

    plt.xticks( range(1, int(max(soln_sizes)+1)) )
    ax2 = ax1.twinx()
    ydata = numpy.array(num_datapoints)
    
    ax2.bar(list(soln_sizes), list(ydata/sum(ydata)), color='grey', alpha=0.33)
    ax2.set_ylabel('Ratio of all instances', color='grey', **axis_font)
    ax2.set_xlim([xlimmin, xlimmax])
    
    #     ax2.set_yscale("log")
    ax2.tick_params('y', colors='grey', labelsize=ytick_font)

    ax1.legend(loc='lower right', frameon=False, prop={'size':axis_font_size})#, handles=ax1, labels=alg_names)
    fig.tight_layout()

    if show_flag:
        plt.show()
    else:
        plt.savefig( './figures/' + ''.join([figurename, froot, '.pdf']), format='pdf', dpi=300)
        plt.savefig( './figures/' + ''.join([figurename, froot, '.png']), format='png', dpi=300)
    plt.close()
    print("Done with plot {}".format(figurename))

In [29]:
show_flag = False
for which_data in [0,1,2]:
    froot = froots[which_data]
    plot_ratios(figurename, froot, all_stats[which_data], indices_identical[which_data], indices_timeout[which_data], show_flag)

 catfish with # data points 526815 
 toboggan with # data points 526815 
Done with plot size-vs-ratio-identical-
 catfish with # data points 470349 
 toboggan with # data points 470349 
Done with plot size-vs-ratio-identical-
 catfish with # data points 445465 
 toboggan with # data points 445465 
Done with plot size-vs-ratio-identical-
